# L8: Evaluation part I

Évaluer les réponses du LLM lorsqu'il y a une seule "bonne réponse"

## Configuration
#### Chargement de la clé OpenAI et des librairies Python

In [ ]:
import os
from openai import OpenAI
import sys
sys.path.append('../..')
import utils
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)

In [ ]:
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

![Evaluation part I](slides/evaluation_part01_01.jpg)

#### Obtenir les produits et catégories pertinents
Voici la liste des produits et catégories qui se trouvent dans le catalogue des produits.

In [ ]:
products_and_category = utils.get_products_and_category()
products_and_category

### Trouver les noms de produits et de catégories pertinents (version 1)
Ceci pourrait être la version qui est en production.

In [ ]:
def find_category_and_product_v1(user_input,products_and_category):

    delimiter = "####"
    system_message = f"""
    Vous recevrez des demandes de service client. La demande de service client 
    sera délimitée par les caractères {delimiter}.
    Affichez une liste d'objets JSON Python, où chaque objet a le format suivant :
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'produits': <a list of products that must be found in the allowed products below>

    Les catégories et les produits doivent être trouvés dans la demande de service client.
    Si un produit est mentionné, il doit être associé à la bonne catégorie dans la 
    liste des produits autorisés ci-dessous. Si aucun produit ou catégorie 
    n'est trouvé, affichez une liste vide.
    

    Énumérez tous les produits qui sont pertinents pour la demande de service client 
    en fonction de leur relation avec le nom du produit et la catégorie du produit.
    Ne partez pas du nom du produit pour supposer des caractéristiques ou des attributs 
    tels que la qualité ou le prix relatifs. 

    Les produits autorisés sont fournis au format JSON. Les clés de chaque élément 
    représentent la catégorie. Les valeurs de chaque élément sont une liste de 
    produits qui se trouvent dans cette catégorie.
    Produits autorisés : {products_and_category}

    """
    
    few_shot_user_1 = """Je veux l'ordinateur le plus cher"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


### Evaluate on some queries

In [ ]:
customer_msg_0 = f"""Quel téléviseur puis-je acheter si je suis à petit budget ?"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

In [ ]:
customer_msg_1 = f"""J'ai besoin d'un chargeur pour mon smartphone."""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)

In [ ]:
customer_msg_2 = f"""
Quels ordinateurs avez-vous ?"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

In [ ]:
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

### Cas de test plus difficiles
Identifiez les demandes trouvées en production, où le modèle ne fonctionne pas comme prévu. 

In [ ]:
customer_msg_4 = f"""
Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap, 
le reflex numérique. Quelles télévisions avez-vous également ?"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,
                                                      products_and_category)
print(products_by_category_4)

### Modifier le prompt pour travailler sur des cas plus complexes

In [ ]:
def find_category_and_product_v2(user_input,products_and_category):
    """
    Ajouté : Ne pas afficher de texte supplémentaire qui n'est pas au format JSON.
    Ajouté un deuxième exemple (pour la méthode de few-shot prompting) où 
    l'utilisateur demande l'ordinateur le moins cher. Dans les deux exemples 
    de few-shot, la réponse affichée est la liste complète des produits uniquement 
    au format JSON.
    """
    delimiter = "####"
    system_message = f"""
    Vous recevrez des demandes de service client. La demande de service client 
    sera délimitée par les caractères {delimiter}.
    Affichez une liste d'objets JSON Python, où chaque objet a le format suivant :
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'produits': <a list of products that must be found in the allowed products below>
    Ne pas afficher de texte supplémentaire qui n'est pas au format JSON.
    Ne rédigez pas de texte explicatif après avoir affiché le JSON demandé.

    Les catégories et les produits doivent être trouvés dans la demande de 
    service client. Si un produit est mentionné, il doit être associé à la 
    bonne catégorie dans la liste des produits autorisés ci-dessous.
    Si aucun produit ou catégorie n'est trouvé, affichez une liste vide.

    Énumérez tous les produits qui sont pertinents pour la demande de service 
    client en fonction de leur relation avec le nom du produit et la catégorie 
    du produit. Ne partez pas du nom du produit pour supposer des caractéristiques 
    ou des attributs tels que la qualité ou le prix relatifs.

    Les produits autorisés sont fournis au format JSON. Les clés de chaque élément 
    représentent la catégorie. Les valeurs de chaque élément sont une liste de produits 
    qui se trouvent dans cette catégorie. Produits autorisés : {products_and_category}
    

    """
    
    few_shot_user_1 = """Je veux l'ordinateur le plus cher. Que recommandez-vous ?"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    few_shot_user_2 = """Je veux l'ordinateur le moins cher. Que recommandez-vous ?"""
    few_shot_assistant_2 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


### Evaluate the modified prompt on the hard tests cases

In [ ]:
customer_msg_3 = f"""
Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap, le reflex numérique.
Quelles télévisions avez-vous également ?"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

### Test de régression : vérifiez que le modèle fonctionne toujours sur les cas de test précédents
Vérifiez que la modification du modèle pour corriger les cas de test difficiles n'affecte pas négativement ses performances sur les cas de test précédents.

In [ ]:
customer_msg_0 = f"""Quel téléviseur puis-je acheter si je suis à petit budget ?"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

### Rassembler l'ensemble de développement pour les tests automatisés

In [ ]:
msg_ideal_pairs_set = [
    
    # eg 0
    {'customer_msg':"""Quel téléviseur puis-je acheter si je suis à petit budget ?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']
        )}
    },

    # eg 1
    {'customer_msg':"""J'ai besoin d'un chargeur pour mon smartphone.""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']
        )}
    },
    # eg 2
    {'customer_msg':f"""Quels ordinateurs avez-vous ?""",
     'ideal_answer':{
           'Computers and Laptops':set(
               ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'
               ])
                }
    },

    # eg 3
    {'customer_msg':f"""Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap, 
     le reflex numérique.
     Quelles télévisions avez-vous également ?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX ProPhone']),
        'Cameras and Camcorders':set(
            ['FotoSnap DSLR Camera']),
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
        }
    }, 
    
    # eg 4
    {'customer_msg':"""Parlez-moi de la télévision CineView, celle en 8K, et de la console Gamesphere, la X.
     Je suis à petit budget, quels ordinateurs avez-vous ?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 8K TV']),
        'Gaming Consoles and Accessories':set(
            ['GameSphere X']),
        'Computers and Laptops':set(
            ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
        }
    },
    
    # eg 5
    {'customer_msg':f"""Quels smartphones avez-vous ?""",
     'ideal_answer':{
           'Smartphones and Accessories':set(
               ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds'
               ])
                    }
    },
    # eg 6
    {'customer_msg':f"""J'ai un petit budget. Pouvez-vous me recommander quelques smartphones ?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartX ProPhone', 'MobiTech Wireless Charger']
        )}
    },

    # eg 7 # this will output a subset of the ideal answer
    {'customer_msg':f"""Quelles consoles de jeux seraient bonnes pour mon ami qui aime les jeux de course ?""",
     'ideal_answer':{
        'Gaming Consoles and Accessories':set([
            'GameSphere X',
            'ProGamer Controller',
            'GameSphere Y',
            'ProGamer Racing Wheel',
            'GameSphere VR Headset'
     ])}
    },
    # eg 8
    {'customer_msg':f"""Quel pourrait être un bon cadeau pour mon ami vidéographe ?""",
     'ideal_answer': {
        'Cameras and Camcorders':set([
        'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera', 'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
        ])}
    },
    
    # eg 9
    {'customer_msg':f""" Je voudrais une machine à remonter le temps en forme de bain à remous.""",
     'ideal_answer': []
    }
    
]


### Évaluer les cas de test en les comparant aux réponses idéales

In [ ]:
import json
def eval_response_with_ideal(response,
                              ideal,
                              debug=False):
    
    if debug:
        print("response")
        print(response)
    
    # json.loads() expects double quotes, not single quotes
    json_like_str = response.replace("'",'"')
    
    # parse into a list of dictionaries
    l_of_d = json.loads(json_like_str)
    
    # special case when response is empty list
    if l_of_d == [] and ideal == []:
        return 1
    
    # otherwise, response is empty 
    # or ideal should be empty, there's a mismatch
    elif l_of_d == [] or ideal == []:
        return 0
    
    correct = 0    
    
    if debug:
        print("l_of_d is")
        print(l_of_d)
    for d in l_of_d:

        cat = d.get('category')
        prod_l = d.get('products')
        if cat and prod_l:
            # convert list to set for comparison
            prod_set = set(prod_l)
            # get ideal set of products
            ideal_cat = ideal.get(cat)
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"did not find category {cat} in ideal")
                    print(f"ideal: {ideal}")
                continue
                
            if debug:
                print("prod_set\n",prod_set)
                print()
                print("prod_set_ideal\n",prod_set_ideal)

            if prod_set == prod_set_ideal:
                if debug:
                    print("correct")
                correct +=1
            else:
                print("incorrect")
                print(f"prod_set: {prod_set}")
                print(f"prod_set_ideal: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("response is a subset of the ideal answer")
                elif prod_set >= prod_set_ideal:
                    print("response is a superset of the ideal answer")

    # count correct over total number of items in list
    pc_correct = correct / len(l_of_d)
        
    return pc_correct

In [ ]:
print(f'Message utilisateur: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'Réponse idéale: {msg_ideal_pairs_set[7]["ideal_answer"]}')


In [ ]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'Réponse: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])

### Exécuter l'évaluation sur tous les cas de test et calculer la fraction des cas qui sont corrects

In [ ]:
# Note, this will not work if any of the api calls time out
score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    print(f"example {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']
    
    # print("Customer message",customer_msg)
    # print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                                      products_and_category)

    
    # print("products_by_category",products_by_category)
    score = eval_response_with_ideal(response,ideal,debug=False)
    print(f"{i}: {score}")
    score_accum += score
    

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"Fraction correct out of {n_examples}: {fraction_correct}")